In [4]:
import cvxpy as cp
import numpy as np
# Define problem parameters
n = 2
c = np.ones((n+1, 1))
b = np.vstack((c, np.zeros((n, 1))))
mu = 0.3 * np.ones((3*n, 1))
h = 0.5 * np.ones((n, 1))
sigma1 = np.diag(h.flatten())
sigma2 = np.diag(h.flatten())
sigma = np.zeros((3*n, 3*n))
sigma[0:n, 0:n] = sigma1 + sigma2
sigma[n:2*n, n:2*n] = sigma1
sigma[2*n:3*n, 2*n:3*n] = sigma2

# Define matrices A, P, Q, and O
P = np.zeros((n+1, n))
P[0, 0] = -1
P[n, n-1] = 1
for i in range(1, n):
    P[i, i-1] = 1
    P[i, i] = -1

Q = np.zeros((n, n))
Q[n-1, n-1] = -1
for i in range(n-1):
    Q[i, i] = -1
    Q[i, i+1] = 1

O = np.zeros((n, n+1))
for i in range(n):
    O[i, i+1] = 1

A = np.zeros((2*n+1, 4*n+1))
A[0:n+1, 0:n+1] = np.eye(n+1) 
A[0:n+1, n+1:2*n+1] = np.zeros((n+1, n))
A[0:n+1, 2*n+1:3*n+1] = P
A[0:n+1, 3*n+1:4*n+1] = np.zeros((n+1, n))
A[n+1:2*n+1, 0:n+1] = O
A[n+1:2*n+1, n+1:2*n+1] = Q
A[n+1:2*n+1, 2*n+1:3*n+1] = np.zeros((n, n))
A[n+1:2*n+1, 3*n+1:4*n+1] = -np.eye(n)

T = 8000

# Define optimization variables
s = cp.Variable(n)
alph = cp.Variable()
bet = cp.Variable(3*n)
T0 = cp.Variable((3*n, 3*n), PSD=True)
T1 = cp.Variable(2*n+1)
X1 = cp.Variable((7*n+2, 7*n+2), PSD=True)

rho = cp.hstack((s[0:n-1] + s[1:n], s[n-1], 0, s[0:n], np.zeros((2, )), np.zeros((n, )))) #1,1,1,2,2,2


# Define objective function
obj = cp.sum(cp.diag(sigma*T0)) + mu.T @ bet + alph
problem = cp.Problem(cp.Minimize(obj))

# Define constraints
constraints = [
    X1[0, 0] == alph,
    X1[1:3*n+1, 0] == 1/2*bet,
    X1[1:3*n+1, 1:3*n+1] == T0,
    X1[3*n+1:7*n+2, 0] == 1/2*rho,
    X1[3*n+1:6*n+2, 1:6*n+2] == np.zeros((3*n+1, 6*n+1)),
    X1[6*n+2:7*n+2, 1:7*n+1] == np.zeros((n, 7*n)),
    cp.sum(s[0:n]) <= T,
    s >= 0
]

# Define semidefinite constraint
Y = cp.Variable((7*n+2, 7*n+2), PSD=True)
for i in range(n):
    T1i = T1[i]
    bi = b[i]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = cp.hstack((-bi,cp.Constant(value=np.zeros((3*n,))), Ai))
    #Bi = cp.vstack((-bi, np.zeros((3*n, 1)), Ai))
    Y += T1i * (Bi.T @ Bi) # 
constraints.append(X1 + Y == cp.Variable((7*n+2, 7*n+2), PSD=True))



d:\python\Lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)


In [ ]:
# check the solver family
problem.solve(verbose=True)

In [10]:
# Solve the problem
problem = cp.Problem(cp.Minimize(obj), constraints)
problem.solve(solver=cp.SCS,verbose=True)

# Print the optimal values
print("Optimal value: ", problem.value)
print("s: ", s.value)
print("alph: ", alph.value)
print("bet: ", bet.value)
print("T0: ", T0.value)
print("T1: ", T1.value)
print("X1: ", X1.value)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 13 08:28:15 PM: Your problem has 818 variables, 9 constraints, and 0 parameters.
(CVXPY) Jun 13 08:28:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 13 08:28:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 13 08:28:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 13 08:28:15 PM: Compiling problem (target solver=SCS).
(CVXPY) Jun 13 08:28:15 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

In [121]:
import cvxpy as cp
import numpy as np
# Define problem parameters
n = 3
c = np.ones((n+1, 1))
b = np.vstack((c, np.zeros((n, 1))))
mu = 0.3 * np.ones((3*n, 1))
h = 0.5 * np.ones((n, 1))
sigma1 = np.diag(h.flatten())
sigma2 = np.diag(h.flatten())
sigma = np.zeros((3*n, 3*n))
sigma[0:n, 0:n] = sigma1 + sigma2
sigma[n:2*n, n:2*n] = sigma1
sigma[2*n:3*n, 2*n:3*n] = sigma2

In [122]:
# Define matrices A, P, Q, and O
P = np.zeros((n+1, n))
P[0, 0] = -1
P[n, n-1] = 1
for i in range(1, n):
    P[i, i-1] = 1
    P[i, i] = -1

Q = np.zeros((n, n))
Q[n-1, n-1] = -1
for i in range(n-1):
    Q[i, i] = -1
    Q[i, i+1] = 1

O = np.zeros((n, n+1))
for i in range(n):
    O[i, i+1] = 1

A = np.zeros((2*n+1, 4*n+1))
A[0:n+1, 0:n+1] = np.eye(n+1) 
A[0:n+1, n+1:2*n+1] = np.zeros((n+1, n))
A[0:n+1, 2*n+1:3*n+1] = P
A[0:n+1, 3*n+1:4*n+1] = np.zeros((n+1, n))
A[n+1:2*n+1, 0:n+1] = O
A[n+1:2*n+1, n+1:2*n+1] = Q
A[n+1:2*n+1, 2*n+1:3*n+1] = np.zeros((n, n))
A[n+1:2*n+1, 3*n+1:4*n+1] = -np.eye(n)

In [123]:
A

array([[ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0., -1.,  1.,  0.,  0., -1., -0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0., -0., -1.]])

In [124]:
T = 8000

# Define optimization variables
s = cp.Variable(n)
alph = cp.Variable()
bet = cp.Variable(3*n)
T0 = cp.Variable((3*n, 3*n), PSD=True)
T1 = cp.Variable(2*n+1)
X1 = cp.Variable((7*n+2, 7*n+2), PSD=True)

rho = cp.hstack((s[0:n-1] + s[1:n], s[n-1], 0, s[0:n], np.zeros((2, )), np.zeros((n, )))) #1,1,1,2,2,2

In [125]:
a = cp.hstack((s[0],s[1]))

In [126]:
rho

Expression(AFFINE, UNKNOWN, (9,))

In [172]:
# Define optimization variables
s = cp.Variable(n)
alph = cp.Variable()
bet = cp.Variable(3*n)
T0 = cp.Variable((3*n, 3*n), PSD=True)
T1 = cp.Variable(2*n+1)
X1 = cp.Variable((7*n+2, 7*n+2), PSD=True)

# Define rho
rho = cp.hstack((s[0:n-1] + s[1:n], s[n-1], 0, s[0:n], np.zeros((2, )), np.zeros((n, ))))

# Define objective function
obj = cp.sum(cp.diag(sigma*T0)) + mu.T @ bet + alph
problem = cp.Problem(cp.Minimize(obj))

# Define constraints
constraints = [
    X1[0, 0] == alph,
    X1[1:3*n+1, 0] == 1/2*bet,
    X1[1:3*n+1, 1:3*n+1] == T0,
    X1[3*n+1:7*n+2, 0] == 1/2*rho,
    X1[3*n+1:6*n+2, 1:6*n+2] == np.zeros((3*n+1, 6*n+1)),
    X1[6*n+2:7*n+2, 1:7*n+1] == np.zeros((n, 7*n)),
    cp.sum(s[0:n]) <= T,
    s >= 0
]

d:\python\Lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 19 times so far.

  warnings.warn(msg, UserWarning)


In [173]:
A

array([[ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0., -1.,  1.,  0.,  0., -1., -0.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0., -0., -1.]])

In [174]:
np.zeros((3*n, 1))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [105]:
Y = cp.Variable((7*n+2, 7*n+2), PSD=True)

In [149]:
ss = bi.T #array([[1.]])


In [152]:
np.zeros((3*n,))

array([0., 0., 0., 0., 0., 0.])

In [165]:
T1i = T1[i]
bi = b[i]
C = A[i, 0:4*n+1]
Ai = C.T
Bi = cp.hstack((cp.Constant(value=np.zeros((3*n,))), bi, Ai))


In [166]:
Bi

Expression(CONSTANT, UNKNOWN, (16,))

In [139]:
A[i, 0:4*n+1] #array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.])

array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.])

In [156]:
A[i, 0:4*n+1]

array([ 1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.])

In [175]:


# Define semidefinite constraint
Y = cp.Variable((7*n+2, 7*n+2), PSD=True)
for i in range(n):
    T1i = T1[i]
    bi = b[i]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = cp.hstack((cp.Constant(value=np.zeros((3*n,))), bi, Ai))
    #Bi = cp.vstack((-bi, np.zeros((3*n, 1)), Ai))
    Y += T1i * (Bi.T @ Bi) # 
constraints.append(X1 + Y == cp.Variable((7*n+2, 7*n+2), PSD=True))

# Solve the problem
problem = cp.Problem(cp.Minimize(obj), constraints)
problem.solve()

# Print the optimal values
print("Optimal value: ", problem.value)
print("s: ", s.value)
print("alph: ", alph.value)
print("bet: ", bet.value)
print("T0: ", T0.value)
print("T1: ", T1.value)
print("X1: ", X1.value)

Optimal value:  4.264612495458194e-06
s:  [7.00023028e-07 5.78739132e-07]
alph:  2.026290839208908e-07
bet:  [2.04152374e-06 2.04152397e-06 2.54537273e-06 2.54537235e-06
 2.54537206e-06 2.54537271e-06]
T0:  [[-4.99801439e-08 -4.98900318e-08 -5.26746527e-08 -5.26745781e-08
  -5.26746711e-08 -5.26745260e-08]
 [-4.98900318e-08 -4.99794268e-08 -5.26746955e-08 -5.26746607e-08
  -5.26746486e-08 -5.26746327e-08]
 [-5.26746527e-08 -5.26746955e-08 -5.87086681e-08 -5.84854911e-08
  -5.84855180e-08 -5.84854453e-08]
 [-5.26745781e-08 -5.26746607e-08 -5.84854911e-08 -5.87090625e-08
  -5.84856571e-08 -5.84856286e-08]
 [-5.26746711e-08 -5.26746486e-08 -5.84855180e-08 -5.84856571e-08
  -5.87094516e-08 -5.84857455e-08]
 [-5.26745260e-08 -5.26746327e-08 -5.84854453e-08 -5.84856286e-08
  -5.84857455e-08 -5.87093965e-08]]
T1:  [-2.17558008 -1.63168297  0.          0.          0.        ]
X1:  [[ 2.02706836e-07  1.02077464e-06  1.02077475e-06  1.27269860e-06
   1.27269832e-06  1.27269822e-06  1.27269851e-0

In [112]:
for i in range(n):
    T1i = T1[i, 0]
    bi = b[i, 0]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = np.vstack((-bi, np.zeros((3*n, 1)), Ai))
    Y += T1i * (Bi @ Bi.T)

IndexError: Too many indices for expression.

In [ ]:
import cvxpy as cp
import numpy as np



# Define objective function and constraints
obj = cp.Minimize(cp.sum(cp.diag(sigma @ T0)) + mu.T @ bet + alph)
constraints = [X1[0, 0] == alph,
               X1[1:3*n+1, 0] == 0.5 * bet,
               X1[1:3*n+1, 1:3*n+1] == T0,
               X1[3*n+1:7*n+2, 0] == 0.5 * rho,
               X1[3*n+1:6*n+2, 1:6*n+2] == np.zeros((3*n+1, 6*n+1)),
               X1[6*n+2:7*n+2, 1:7*n+2] == np.zeros((n, 7*n+1)),
               cp.sum(s[0:n]) <= T,
               s >= 0]

for i in range(n):
    T1i = T1[i, 0]
    bi = b[i, 0]
    C = A[i, 0:4*n+1] # Fix the index error here
    Ai = C.T
    Bi = np.vstack((-bi, np.zeros((3*n, 1)), Ai))
    constraints.append(T1i * (Bi @ Bi.T) >> 0)


# Solve the problem
prob = cp.Problem(obj, constraints)
prob.solve()

# Print the optimal values
print("Optimal value: ", prob.value)
print("s: ", s.value)
print("alph: ", alph.value)
print("bet: ", bet.value)
print("T0: ", T0.value)
print("T1: ", T1.value)
print("X1: ", X1.value)

ValueError: setting an array element with a sequence.

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]
[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


In [ ]:
# Define objective function and constraints
obj = cp.Minimize(cp.sum(cp.diag(sigma @ T0)) + mu.T @ bet + alph)
constraints = [X1[0, 0] == alph,
               X1[1:3*n+1, 0] == 0.5 * bet,
               X1[1:3*n+1, 1:3*n+1] == T0,
               X1[3*n+1:7*n+2, 0] == 0.5 * rho,
               X1[3*n+1:6*n+2, 1:6*n+2] == cp.zeros((3*n+1, 6*n+1)),
               X1[6*n+2:7*n+2, 1:7*n+2] == cp.zeros((n, 7*n+1)),
               cp.sum(s[0:n]) <= T,
               s >= 0]

for i in range(n):
    T1i = T1[i, 0]
    bi = b[i, 0]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = cp.vstack((-bi, cp.zeros((3*n, 1)), Ai))
    constraints.append(T1i * (Bi @ Bi.T) >> 0)

# Solve the problem
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.MOSEK)

# Print the optimal values
print("Optimal value: ", prob.value)
print("s: ", s.value)
print("alph: ", alph.value)
print("bet: ", bet.value)
print("T0: ", T0.value)
print("T1: ", T1.value)
print("X1: ", X1.value)


ValueError: setting an array element with a sequence.

import cvxpy as cp
import numpy as np

# Define problem parameters
n = 2
c = np.ones((n+1, 1))
b = np.vstack((c, np.zeros((n, 1))))
mu = 0.3 * np.ones((3*n, 1))
h = 0.5 * np.ones((n, 1))
sigma1 = np.diag(h.flatten())
sigma2 = np.diag(h.flatten())
sigma = np.zeros((3*n, 3*n))
sigma[0:n, 0:n] = sigma1 + sigma2
sigma[n:2*n, n:2*n] = sigma1
sigma[2*n:3*n, 2*n:3*n] = sigma2

# Define matrices A, P, Q, and O
P = np.zeros((n+1, n))
P[0, 0] = -1
P[n, n-1] = 1
for i in range(1, n):
    P[i, i-1] = 1
    P[i, i] = -1

Q = np.zeros((n, n))
Q[n-1, n-1] = -1
for i in range(n-1):
    Q[i, i] = -1
    Q[i, i+1] = 1

O = np.zeros((n, n+1))
for i in range(n):
    O[i, i+1] = 1

A = np.zeros((2*n+1, 4*n+2))
A[0:n+1, 0:n+1] = np.eye(n+1)
A[0:n+1, n+1:2*n+1] = np.zeros((n+1, n))
A[0:n+1, 2*n+1:3*n+1] = P
A[0:n+1, 3*n+1:4*n+1] = np.zeros((n+1, n))
A[n+1:2*n+1, 0:n+1] = O
A[n+1:2*n+1, n+1:2*n+1] = Q
A[n+1:2*n+1, 2*n+1:3*n+1] = np.zeros((n, n))
A[n+1:2*n+1, 3*n+1:4*n+1] = -np.eye(n)

T = 8000

# Define optimization variables
s = cp.Variable(n)
alph = cp.Variable()
bet = cp.Variable(3*n)
T0 = cp.Variable((3*n, 3*n), PSD=True)
T1 = cp.Variable(2*n+1)
X1 = cp.Variable((7*n+2, 7*n+2), PSD=True)

rho = np.hstack((s[0:n-1] + s[1:n], s[n], 0, s[0:n], s[2:n], np.zeros((2, 1)), np.zeros((n, 1))))

# Define objective function and constraints
obj = cp.Minimize(cp.sum(cp.diag(sigma @ T0)) + mu.T @ bet + alph)
constraints = [X1[0, 0] == alph,
               X1[1:3*n+1, 0] == 0.5 * bet,
               X1[1:3*n+1, 1:3*n+1] == T0,
               X1[3*n+1:7*n+2, 0] == 0.5 * rho,
               X1[3*n+1:6*n+2, 1:6*n+2] == np.zeros((3*n+1, 6*n+1)),
               X1[6*n+2:7*n+2, 1:7*n+2] == np.zeros((n, 7*n+1)),
               cp.sum(s[0:n]) <= T,
               s >= 0]

for i in range(n):
    T1i = T1[i, 0]
    bi = b[i, 0]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = np.vstack((-bi, np.zeros((3*n, 1)), Ai))
    constraints.append(T1i * (Bi @ Bi.T) >> 0)

# Solve the problem
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.MOSEK)

# Print the optimal values
print("Optimal value: ", prob.value)
print("s: ", s.value)
print("alph: ", alph.value)
print("bet: ", bet.value)
print("T0: ", T0.value)
print("T1: ", T1.value)
print("X1: ", X1.value)

In [ ]:
import cvxpy as cp
import numpy as np

# Define problem parameters
n = 2
c = np.ones((n+1, 1))
b = np.vstack((c, np.zeros((n, 1))))
mu = 0.3 * np.ones((3*n, 1))
h = 0.5 * np.ones((n, 1))
sigma1 = np.diag(h.flatten())
sigma2 = np.diag(h.flatten())
sigma = np.zeros((3*n, 3*n))
sigma[0:n, 0:n] = sigma1 + sigma2
sigma[n:2*n, n:2*n] = sigma1
sigma[2*n:3*n, 2*n:3*n] = sigma2

# Define matrices A, P, Q, and O
P = np.zeros((n+1, n))
P[0, 0] = -1
P[n, n-1] = 1
for i in range(1, n):
    P[i, i-1] = 1
    P[i, i] = -1

Q = np.zeros((n, n))
Q[n-1, n-1] = -1
for i in range(n-1):
    Q[i, i] = -1
    Q[i, i+1] = 1

O = np.zeros((n, n+1))
for i in range(n):
    O[i, i+1] = 1

A = np.zeros((2*n+1, 4*n+2))
A[0:n+1, 0:n+1] = np.eye(n+1)
A[0:n+1, n+1:2*n+1] = np.zeros((n+1, n))
A[0:n+1, 2*n+1:3*n+1] = P
A[0:n+1, 3*n+1:4*n+1] = np.zeros((n+1, n))
A[n+1:2*n+1, 0:n+1] = O
A[n+1:2*n+1, n+1:2*n+1] = Q
A[n+1:2*n+1, 2*n+1:3*n+1] = np.zeros((n, n))
A[n+1:2*n+1, 3*n+1:4*n+1] = -np.eye(n)

T = 8000

# Define optimization variables
s = cp.Variable(n)
alph = cp.Variable()
bet = cp.Variable(3*n)
T0 = cp.Variable((3*n, 3*n), PSD=True)
T1 = cp.Variable(2*n+1)
X1 = cp.Variable((7*n+2, 7*n+2), PSD=True)

rho = np.hstack((s[0:n-1] + s[1:n], s[n-1:n], np.array([0]), s[0:n-1], s[2:n], np.zeros((2,)), np.zeros((n,))))

Y = np.zeros((7*n+2, 7*n+2))
Z = X1 + Y
# Define optimization problem
obj = cp.Minimize(cp.sum(cp.diag(sigma @ T0)) + mu.T @ bet + alph)
constraints = [
    X1[0, 0] == alph,
    X1[1:3*n+1, 0] == 1/2 * bet,
    X1[1:3*n+1, 1:3*n+1] == T0,
    #X1[3*n+1:7*n+2, 0] == 1/2 * rho,
    X1[3*n+1:6*n+2, 1:6*n+2] == np.zeros((3*n+1, 6*n+1)),
    X1[6*n+2:7*n+2, 1:7*n+2] == np.zeros((n, 7*n+1)),
    cp.sum(s[0:n]) <= T,
    s >= 0,
]

for i in range(n):
    T1i = T1[i, 0]
    bi = b[i, 0]
    C = A[i, 0:4*n+1]
    Ai = C.T
    Bi = np.vstack((-bi, np.zeros((3*n, 1)), Ai))
    Y += T1i * (Bi @ Bi.T)

prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.MOSEK)

# Print the optimal values
print("Optimal value: ", prob.value)
print("Optimal s: ", s.value)
print("Optimal alph: ", alph.value)
print("Optimal bet: ", bet.value)
print("Optimal T0: ", T0.value)
print("Optimal T1: ", T1.value)
print("Optimal X1: ", X1.value)

IndexError: Too many indices for expression.